# imports

In [80]:
import segyio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import display
from segysak.segy import segy_loader, well_known_byte_locs, segy_writer, segy_header_scrape
from ipywidgets import interactive
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# CA coordinate extraction

In [ ]:
df = pd.read_csv(r'C:\Users\AslanRzayev\Desktop\workspace\Projects\eilink\kh_static\data\df_bal_ne 3.csv')
df

In [ ]:
df_ca_bal8 = df[(df['FORMATION_up'] == 'Balakhany VIII') & (df['field'] == 'CENTRAL AZERI')]
df_ca_bal8_coordinates = {'x_min': df_ca_bal8['X_traj'].min(), 
                'x_max': df_ca_bal8['X_traj'].max(),
                'y_min': df_ca_bal8['Y_traj'].min(),
                'y_max': df_ca_bal8['Y_traj'].max(),
                'z_min': df_ca_bal8['TVD_SCS'].min(),
                'z_max': df_ca_bal8['TVD_SCS'].max()              
                }
df_ca_bal8_coordinates


In [ ]:
df_ca_bal10 = df[(df['FORMATION_up'] == 'Balakhany X') & (df['field'] == 'CENTRAL AZERI')]
df_ca_bal10_coordinates = {'x_min': df_ca_bal10['X_traj'].min(), 
                'x_max': df_ca_bal10['X_traj'].max(),
                'y_min': df_ca_bal10['Y_traj'].min(),
                'y_max': df_ca_bal10['Y_traj'].max(),
                'z_min': df_ca_bal10['TVD_SCS'].min(),
                'z_max': df_ca_bal10['TVD_SCS'].max()              
                }
df_ca_bal10_coordinates

# SEGY file

In [ ]:
v = segyio.open(r"C:\Users\NadirZeynalli\Desktop\SPP\GR_cube\GR_cube_HDOBN_Z.sgy")

v.header[0]

In [ ]:
print(v.xlines)
print(v.ilines)
print(v.samples)

In [ ]:
print(len(v.xlines))
print(len(v.ilines))
print(len(v.samples))

In [ ]:
v.iline[v.ilines[600]][100]

In [ ]:
plt.figure(figsize=(2, 15))
plt.plot(v.iline[v.ilines[600]][100],v.samples)
plt.gca().invert_yaxis()

In [ ]:
plt.figure(figsize=(21,10))
plt.imshow(v.iline[v.ilines[5873]].T, vmin=0, vmax=100)
# plt.savefig(r"C:\Users\eiGroup - Nadir\Desktop\GR_cube.jpg")
plt.yticks(ticks=plt.yticks()[0][1:], labels=[str(int(ytick) + 1380) for ytick in plt.yticks()[0][1:]])
plt.show()

In [88]:
# def plot_slice(slice_index):
#     plt.figure(figsize=(21,13))
#     plt.imshow(v.iline[v.ilines[slice_index]].T, vmin=0, vmax=100)
#     plt.yticks(ticks=plt.yticks()[0][1:], labels=[str(int(ytick) + 1380) for ytick in plt.yticks()[0][1:]])
#     plt.show()

# slice_slider = widgets.IntSlider(min=600, max=6600, step=1, value=600, description='Inline number')
# interactive_plot = interactive(plot_slice, slice_index=slice_slider)
# interactive_plot

# Well trajectory

In [ ]:
import lasio

las = lasio.read(r"C:\Users\NadirZeynalli\Desktop\SPP\GR_cube\well_trajectories2\K01.las")

k01 = las.df().reset_index()[['X','Y','TVD','DEPTH']]
k01

## interpolation

In [ ]:
from scipy.interpolate import CubicSpline

df = k01

new_tvd = np.arange(df['TVD'].min(), df['TVD'].max() + 1)

cs_x = CubicSpline(df['TVD'], df['X'])
cs_y = CubicSpline(df['TVD'], df['Y'])
cs_depth = CubicSpline(df['TVD'], df['DEPTH'])

new_x = cs_x(new_tvd)
new_y = cs_y(new_tvd)
new_depth = cs_depth(new_tvd)

interpolated_df = pd.DataFrame({'TVD': new_tvd, 'X': new_x, 'Y': new_y, 'DEPTH':new_depth})

interpolated_df = interpolated_df[interpolated_df['TVD'] >= 1380][interpolated_df['TVD'] <= 4250]
interpolated_df

In [138]:
interpolated_df.rename(columns={
    'X':'CDP_X',
    'Y':'CDP_Y'
}, inplace=True)

# DataFrame of SEGY

In [ ]:
df_cube = pd.read_parquet(r"C:\Users\NadirZeynalli\Desktop\SPP\GR_cube\GR_cube_HDOBN_Z.gzip")
df_cube

In [140]:
df_cube = df_cube[[
    'CDP_X', 
    'CDP_Y', 
    'INLINE_3D', 
    'CROSSLINE_3D'
]]

df_cube['CDP_X'] /= 100
df_cube['CDP_Y'] /= 100

## Closest coordinate

In [ ]:
from scipy.spatial import cKDTree

df1 = interpolated_df
df2 = df_cube

tree = cKDTree(df2[['CDP_X', 'CDP_Y']])

distances, indices = tree.query(df1[['CDP_X', 'CDP_Y']])

closest_points = df2.iloc[indices].reset_index(drop=True)

result = pd.concat([df1.reset_index(drop=True), closest_points], axis=1)

result

# GR extract

In [ ]:
v.ilines

In [ ]:
def GR_extract(iline, xline, sample):
    iline = iline
    xline = xline - 1400
    sample = sample - 1380

    return v.iline[v.ilines[iline]][xline][sample]

result['GR_center'] = result.apply(lambda row: GR_extract(row['INLINE_3D'].astype(int), row['CROSSLINE_3D'].astype(int), row['TVD'].astype(int)), axis=1)
# result['GR1'] = result.apply(lambda row: GR_extract(row['INLINE_3D'].astype(int)+1, row['CROSSLINE_3D'].astype(int), row['TVD'].astype(int)), axis=1)
# result['GR2'] = result.apply(lambda row: GR_extract(row['INLINE_3D'].astype(int), row['CROSSLINE_3D'].astype(int)+1, row['TVD'].astype(int)), axis=1)
# result['GR_center'] = result.apply(lambda row: GR_extract(row['INLINE_3D'].astype(int), row['CROSSLINE_3D'].astype(int)+1, row['TVD'].astype(int)), axis=1)
result

In [ ]:
ax = result.plot(x='GR_center', y='DEPTH', figsize=(3, 20))
ax.invert_yaxis()
plt.grid(True)
plt.show()

# Others (not needed)

In [ ]:
df_cube_ca_bal8 = df_cube[(df_cube['CDP_X'] / 100 >= df_ca_bal8_coordinates['x_min']) &
                          (df_cube['CDP_X'] / 100 <= df_ca_bal8_coordinates['x_max']) &
                          (df_cube['CDP_Y'] / 100 >= df_ca_bal8_coordinates['y_min']) &
                          (df_cube['CDP_Y'] / 100 <= df_ca_bal8_coordinates['y_max']) 
                          ]

df_cube_ca_bal8 = df_cube_ca_bal8[['CDP_X', 'CDP_Y', 'INLINE_3D', 'CROSSLINE_3D']]
df_cube_ca_bal8['CDP_X'] = df_cube_ca_bal8['CDP_X'] / 100
df_cube_ca_bal8['CDP_Y'] = df_cube_ca_bal8['CDP_Y'] / 100

ca_bal8_inlines = df_cube_ca_bal8['INLINE_3D'].unique().tolist()
ca_bal8_crosslines = df_cube_ca_bal8['CROSSLINE_3D'].unique().tolist()

ca_bal8_inlines.sort()
ca_bal8_crosslines.sort()

print(ca_bal8_inlines)
print(len(ca_bal8_inlines))

print(len(ca_bal8_crosslines))
print(ca_bal8_crosslines)


df_cube_ca_bal8.sort_values(by=['INLINE_3D', 'CROSSLINE_3D'], inplace=True)

df_cube_ca_bal8['INLINE_3D_index'] = df_cube_ca_bal8['INLINE_3D'] - df_cube_ca_bal8['INLINE_3D'].min()
df_cube_ca_bal8['CROSSLINE_3D_index'] = df_cube_ca_bal8['CROSSLINE_3D'] - df_cube_ca_bal8['CROSSLINE_3D'].min()

df_cube_ca_bal8 = df_cube_ca_bal8[['CDP_X', 'CDP_Y', 'INLINE_3D', 'INLINE_3D_index', 'CROSSLINE_3D', 'CROSSLINE_3D_index']]

df_cube_ca_bal8['Z_index'] = np.nan
df_cube_ca_bal8['GR_value'] = np.nan

df_cube_ca_bal8

In [ ]:
# Original code for the whole gr cube

volume_samples = []

dt = 4
iline_range = [-1, 10000]
xline_range = [-1, 10000]
time_range  = [-1, 10000]
iline_xline_spacing = 1

mask_il = (iline_range[0] <= v.ilines) & (v.ilines <= iline_range[-1])
mask_xl = (xline_range[0] <= v.xlines) & (v.xlines <= xline_range[-1])
mask_t  = (time_range[0] <= v.samples) & (v.samples <= time_range[1])



for il in tqdm(v.ilines[mask_il]):
    v_slice = v.iline[il][mask_xl, :][:, mask_t][::iline_xline_spacing, ::int(dt / (v.samples[1] - v.samples[0]))]

    volume_samples.append(v_slice)
    
volume_samples = np.array(volume_samples)

In [ ]:
volume_samples = []

dt = 4
iline_range = [-1, 10000]
xline_range = [-1, 10000]
time_range  = [-1, 10000]
iline_xline_spacing = 1

mask_il = (ca_bal8_inlines[0] <= v.ilines) & (v.ilines <= ca_bal8_inlines[-1])
mask_xl = (ca_bal8_crosslines[0] <= v.xlines) & (v.xlines <= ca_bal8_crosslines[-1])
mask_t  = (time_range[0] <= v.samples) & (v.samples <= time_range[1])



for il in tqdm(v.ilines[mask_il]):
    v_slice = v.iline[il][mask_xl, :][:, mask_t][::iline_xline_spacing, ::int(dt / (v.samples[1] - v.samples[0]))]

    volume_samples.append(v_slice)
    
volume_samples = np.array(volume_samples)

In [ ]:
volume_samples.shape

In [ ]:
df_inlines_xlines = df_cube_ca_bal8[[ 'INLINE_3D', 'CROSSLINE_3D']].drop_duplicates()
df_inlines_xlines.sort_values(by=['INLINE_3D', 'CROSSLINE_3D'], inplace=True)
df_inlines_xlines

In [ ]:
inline_crossline_dict = {}

for index, row in df_inlines_xlines.iterrows():
    inline_3d = row['INLINE_3D']
    crossline_3d = row['CROSSLINE_3D']
    
    # Check if the INLINE_3D already exists in the dictionary
    if inline_3d in inline_crossline_dict:
        # If it exists, append the CROSSLINE_3D to the existing list
        inline_crossline_dict[inline_3d].append(crossline_3d)
    else:
        # If it doesn't exist, create a new list with the CROSSLINE_3D as the first element
        inline_crossline_dict[inline_3d] = [crossline_3d]

In [ ]:
inline_crossline_dict

In [ ]:
time_range  = [-1, 10000]
iline_xline_spacing = 1


mask_t  = (time_range[0] <= v.samples) & (v.samples <= time_range[1])

for i in inline_crossline_dict.keys():
    for j in inline_crossline_dict[i]:
        mask_il = (v.ilines == i)
        mask_xl = (v.xlines == j)
        
        # mask_il = (ca_bal8_inlines[i] = v.ilines) & (v.ilines <= ca_bal8_inlines[-1])
        # mask_xl = (ca_bal8_crosslines[] = v.xlines) & (v.xlines <= ca_bal8_crosslines[-1])
        
        v_slice = v.iline[mask_il][mask_xl, :][:, mask_t][::iline_xline_spacing, ::int(dt / (v.samples[1] - v.samples[0]))]
        # volume_samples.append(v_slice)
        # v_z_slice = v.iline[i][j, :][:, mask_t][::iline_xline_spacing, ::int(dt / (v.samples[1] - v.samples[0]))]
        # v_val_slice = v.iline[i][j, :][:, mask_t][::iline_xline_spacing, ::int(dt / (v.samples[1] - v.samples[0]))]
        # print(v_slice)
        # print(v_z_slice)
        # print(v_val_slice)

In [ ]:
volume_samples = np.array(volume_samples)


In [ ]:
volume_samples.shape

In [ ]:
volume_samples = []

dt = 4
iline_range = [-1, 10000]
xline_range = [-1, 10000]
time_range  = [-1, 10000]
iline_xline_spacing = 1

mask_il = (iline_range[0] <= v.ilines) & (v.ilines <= iline_range[1])
mask_xl = (xline_range[0] <= v.xlines) & (v.xlines <= xline_range[1])
mask_t  = (time_range[0] <= v.samples) & (v.samples <= time_range[1])

for il in tqdm(v.ilines[mask_il]):
    print(il)
    # v_slice = v.iline[il][mask_xl, :][:, mask_t][::iline_xline_spacing, ::int(dt / (v.samples[1] - v.samples[0]))]

    # volume_samples.append(v_slice)
    
# volume_samples = np.array(volume_samples)

In [ ]:
import concurrent.futures
from tqdm import tqdm

volume_samples = []

dt = 4
iline_range = [-1, 10000]
xline_range = [-1, 10000]
time_range  = [-1, 10000]
iline_xline_spacing = 1

mask_il = (iline_range[0] <= v.ilines) & (v.ilines <= iline_range[1])
mask_xl = (xline_range[0] <= v.xlines) & (v.xlines <= xline_range[1])
mask_t  = (time_range[0] <= v.samples) & (v.samples <= time_range[1])

# Define a function to process each iline
def process_il(il):
    v_slice = v.iline[il][mask_xl, :][:, mask_t][::iline_xline_spacing, ::int(dt / (v.samples[1] - v.samples[0]))]
    return v_slice

# Use ThreadPoolExecutor to parallelize the loop
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Use tqdm to display a progress bar
    futures = [executor.submit(process_il, il) for il in tqdm(v.ilines[mask_il])]
    
    for future in concurrent.futures.as_completed(futures):
        volume_samples.append(future.result())



In [ ]:
volume_samples = np.array(volume_samples)


In [ ]:
samples_lst = v.samples.tolist()
samples_lst

In [ ]:
df_samples_gr = pd.DataFrame(columns=['inline', 'crossline', 'z', 'value'])

for i in (ca_bal8_inlines):
    for j in (ca_bal8_crosslines):
        for k in (range(len(samples_lst))):

            if(v.gather[i, j].T[k] != 0):
                print(i, j, k)
                
                # Create a dictionary with the values
                data = {
                    'inline': i,
                    'crossline': j,
                    'z': k,
                    'value': v.gather[i, j].T[k][0]
                }
                print(data)
                # df_one_sample = pd.DataFrame(data)
                # Append the dictionary to the DataFrame
                # df_samples_gr = pd.concat([df_samples_gr, data], ignore_index=False)
                
                df_samples_gr = pd.concat([df_samples_gr, pd.DataFrame(data, index=[0])], ignore_index=True)


                
print(df_samples_gr)


In [ ]:
import concurrent.futures
import pandas as pd

# Define your DataFrame columns
df_samples_gr = pd.DataFrame(columns=['inline', 'crossline', 'z', 'value'])

# Define your function to process a single combination of i, j, k
def process_combination(i, j, k):
    if v.gather[i, j].T[k] != 0:
        data = {
            'inline': i,
            'crossline': j,
            'z': k,
            'value': v.gather[i, j].T[k][0]
        }
        print(data)
        
        return data
    else:
        return None

# Define your main function
def main():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i in ca_bal8_inlines:
            for j in ca_bal8_crosslines:
                for k in range(len(samples_lst)):
                    futures.append(executor.submit(process_combination, i, j, k))
        
        for future in concurrent.futures.as_completed(futures):
            data = future.result()
            if data:
                df_samples_gr = df_samples_gr.append(data, ignore_index=True)

if __name__ == "__main__":
    main()
    print(df_samples_gr)


In [ ]:
v.xline

In [ ]:
(v.xline[1500]).shape

In [ ]:
v.gather[600 , 1400].T[550]

In [ ]:
df_samples_gr = pd.DataFrame(columns=['i', 'j', 'k', 'value'])

for i in (v.ilines):
    for j in (v.xlines):
        for k in (range(len(v.samples))):
            i = int(i)
            j = int(j)
            k = int(k)
            
            if(v.gather[i, j].T[k] != 0):
                print(i, j, k)
            

In [ ]:
v.ilines

In [ ]:
ilines_lst = v.ilines.tolist()
ilines_lst